# Metadata json to netCDF converter
Converting a jason metadata template file to a comtomized netCDF file, which contains only the header without data. 
From the template, users could add the attributes and change the value for each attribute for dimensions, global attributes and variables. 

In [ ]:
# Import modules
import os 
import json
from netCDF4 import Dataset

In [ ]:
# Define output nc/json filename
#output_file = "point_mooring"
#output_file = "profile_ctd"
#output_file = "profile_sonde"
#output_file = "satellite_l2"
#output_file = "satellite_L3_L4"
output_file = "trajectory_drifter"

In [ ]:
# Listing all the template JSON files in smpl/ sub-directory
parentdir = os.getcwd()
jpath = os.path.join(parentdir, 'smpl') 
smpl_dir_list = os.listdir(jpath)
x=[print(i,smpl) for i, smpl in enumerate(smpl_dir_list)]

In [ ]:
# Choose the template JSON file from the list by entering the index value.
ind = input("Choose the sample_json file by entering the index value: ")
# example_json = smpl_dir_list[int(ind)]
smpl_json = os.path.join(jpath,smpl_dir_list[int(ind)])
print(smpl_json)

In [ ]:
# Read the json file
with open(smpl_json, 'r') as f:
    mData = json.loads(f.read())

In [ ]:
# Print the sample json file
json_formatted_str = json.dumps(mData, indent=2)
print(json_formatted_str)

In [ ]:
# function to add and remove key:value pair
def add_rem_attri(grp_dict, isValGrp = ''):
    # Add the new attribute
    newKey = input("++ Add a new attribute KEY? (KEY-Name/Enter(no)): ")
    while newKey != '':
        if isValGrp == 'var':
            newValtmpl = input("Enter the template(example) var name: ")
            grp_dict[newKey]= grp_dict[newValtmpl]
        else:
            grp_dict[newKey]= ''
            
        newKey = input("++ Add another new attribute KEY? (KEY-Name/Enter(no)): ")
            
    # Remove the example_var variables
    delKey = input("-- Delete a attribute KEY? (KEY-Name/Enter(no)): ")
    while delKey != '':
        if delKey in grp_dict.keys(): 
            del grp_dict[delKey]

        delKey = input("-- Delete another KEY? (KEY-Name/Enter(no)): ")
        
    print([key for key in grp_dict.keys()])


In [ ]:
# A recursive function to change the attribute values.
def change_val(grp_attr):
    for key, value in grp_attr.items():
        if (type(value) is dict):
            #print("\n", "GROUP NAME: " + key)
            x = input("#### Change the attributes of '" + key + "'? (Y/Enter(no)): ")
            if x.lower() == 'y':
                # Print the json file
                json_formatted_str = json.dumps(value, indent=2)
                print(json_formatted_str)
                change_val(value)
        else:
            #print ('\t', key, ' = ', value)
            x = input("--> Change the '" + key + "' value? (Y/Enter(no)): ")
            if x.lower() == 'y':
                type_str = input("  -->> Enter the data type (int, float, list, or default('str')): ")
                if type_str ==  "int":
                    grp_attr[key] = int(input("Enter new value for '" + key + "': "))
                elif type_str ==  "float":
                    grp_attr[key] = float(input("Enter new value for '" + key + "': "))
                elif type_str ==  "list":
                    list_type = input("  -->> Enter the LIST type (int, float, or default('str')): ")
                    strList = input("Enter new value for '" + key + "': ").split(',')
                    if list_type ==  "int":
                        grp_attr[key] = list(map(int,strList))
                    elif list_type ==  "float":
                        grp_attr[key] = list(map(float,strList))
                    else:
                        grp_attr[key] = strList
                else:
                    grp_attr[key] = input("Enter new value for '" + key + "': ")

# Review and change "dimensions"

In [ ]:
# dimensions fields
dims = mData['dimensions']
dims

In [ ]:
# Add and/or remove Key:Value pairs of dimensions
add_rem_attri(dims)

In [ ]:
# Review and change the attribute values 
change_val(dims)

In [ ]:
print(dims)

# Review and change "global attributes"

In [ ]:
# Global attribute dict
glb_attr = mData['global_attributes']
glb_attr

In [ ]:
# Add and/or remove Key:Value pairs of dimensions
add_rem_attri(glb_attr)

In [ ]:
# Loop through the global attributes, change the attribute value as needed.
change_val(glb_attr)

In [ ]:
glb_attr

# Review and change "variables"

In [ ]:
# Variables group
var_attr = mData['variables']
#var_attr
print([key for key in var_attr.keys()])

In [ ]:
# Add and/or remove Key:Value pairs of dimensions
add_rem_attri(var_attr, 'var')

In [ ]:
# Loop through the variables
change_val(var_attr)

In [ ]:
# reviewing all the variables
var_attr

# Write the new metadata to netCDF and json files to out_dir directory

In [ ]:
# Define the output netCDF file path
oPath = os.path.join(parentdir, 'out_dir') 
empty_nc = os.path.join(oPath, output_file + '.nc')
empty_nc

In [ ]:
if not os.path.exists(oPath):
    os.makedirs(oPath)

In [ ]:
# Create an empty netCDF file
with Dataset(empty_nc, "w") as nc:
    # Add dimensions
    for dim_name, dim_size in mData["dimensions"].items():
        nc.createDimension(dim_name, dim_size)

    # Add variables and their attributes
    for var_name, var_attrs in mData["variables"].items():
        var = nc.createVariable(var_name, var_attrs["type"], var_attrs["dimensions"])
        for attr_name, attr_value in var_attrs["attributes"].items():
            setattr(var, attr_name, attr_value)

    # Add global attributes
    for global_attr_name, global_attr_value in mData["global_attributes"].items():
        setattr(nc, global_attr_name, global_attr_value)

print(f"Empty netCDF file '{empty_nc}' created successfully with embedded JSON metadata attributes.")

In [ ]:
# Output the modified metadata dict to a json file.
out_json = os.path.join(oPath, output_file + '.json')
with open(out_json, 'w', encoding='utf-8') as f:
    json.dump(mData, f, ensure_ascii=False, indent=4)

In [ ]:
# system call to display the customized meta info from the netCDF. 
!ncdump -h {empty_nc}